In [ ]:
import asyncio
import json
import logging
import os
from datetime import datetime, timezone
from logging import INFO
import google.generativeai as genai

from dotenv import load_dotenv

from graphiti_core import Graphiti
from graphiti_core.nodes import EpisodeType
from graphiti_core.search.search_config_recipes import NODE_HYBRID_SEARCH_RRF


# Configure logging
logging.basicConfig(
    level=INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
logger = logging.getLogger(__name__)


load_dotenv()

#set-up Neo4j and Graphiti 

neo4j_uri = os.environ.get('NEO4J_URI', 'bolt://localhost:7687')
neo4j_user = os.environ.get('NEO4J_USER', 'neo4j')
neo4j_password = os.environ.get('NEO4J_PASSWORD', 'password')

#Set-up Gemini 

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel("gemini-2.5-flash")

class ChatMemory:
    def __init__(self, uri, user, password):
        self.driver = AsyncGraphDatabase.driver(uri, auth=(user, password))
        self.graphiti = Graphiti(uri=uri, user=user, password=password)

    async def save_message_as_episode(self, user_id: str, text: str, sender: str):
        timestamp = datetime.now(timezone.utc).isoformat()

        await self.graphiti.add_episode(
            external_user_id=user_id,
            content=text,
            sender=sender,
            type=EpisodeType.chat,
            timestamp=timestamp,
        )


async def run_conversation():
    memory = ChatMemory(neo4j_uri, neo4j_user, neo4j_password)

    user_id = "user-123"
    user_input = "I need help with my order"

    await memory.save_message_as_episode(user_id, user_input, sender="user")

    response = gemini_model.generate_content(user_input)
    reply = response.text.strip()

    await memory.save_message_as_episode(user_id, reply, sender="agent")

    print("Gemini Reply:", reply)


    asyncio.run(run_conversation())


if __name__ == "__main__":
    asyncio.run(run_conversation())


/Users/mahmoud/agent-with-graphiti/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!pip install genai

  Obtaining dependency information for genai from https://files.pythonhosted.org/packages/de/c7/2a79b24610e15ecc6d7c421cb7a6701decc0402011099b08659a3a76196a/genai-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for ipython<9.0.0,>=8.10.0 from https://files.pythonhosted.org/packages/91/d0/274fbf7b0b12643cbbc001ce13e6a5b1607ac4929d1b11c72460152c9fc3/ipython-8.37.0-py3-none-any.whl.metadata
  Obtaining dependency information for openai<0.28.0,>=0.27.0 from https://files.pythonhosted.org/packages/f1/1f/3a0cb7d172f451b2ca8bf65d9196aa3b6878c010d461257c621e4bd48cad/openai-0.27.10-py3-none-any.whl.metadata
  Obtaining dependency information for tabulate<0.10.0,>=0.9.0 from https://files.pythonhosted.org/packages/40/44/4a5f08c96eb108af5cb50b41f76142f0afa346dfa99d5296fe7202a11854/tabulate-0.9.0-py3-none-any.whl.metadata
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency i